# Base Case: Circular Orbit of Radius 1 AU, Period 1 Year

Create synthetic data for the simplest base case: a circular orbit of radius 1.

Can think of this as approximating the earth: radius = 1 AU, period = 1 year, mass of sun $m_0$ = 1 solar mass

\begin{align}
x(t) &= \cos(\omega t) \\
y(t) &= \sin(\omega t) \\
\omega &= 2 \pi
\end{align}

Taking two derivatives
\begin{align}
\ddot{x}(t) = -\omega^2 x(t)\\
\ddot{y}(t) = -\omega^2 y(t)
\end{align}

Equating the acceleration to $G \cdot m_0$, we can see that in these units the gravitational constant $G$ is
$$G = 4 \pi^2$$

In [1]:
# Library imports
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Aliases
keras = tf.keras

# Local imports
import utils

In [2]:
# Grow GPU memory
utils.gpu_grow_memory()

In [3]:
# The angular velocity of the earth in the (AU, years) coordinate units
omega_earth = 2.0 * np.pi

# The gravitational constant in units of (AU, years, solar_mass)
G = omega_earth**2

# The mass of the sun in this coordinate system
m0 = 1.0

# The gravitational coefficient G*(m0 + m1) for the restricted two body problem in these units
mu = G * (m0)

In [43]:
q0.shape

(3651, 2)

In [128]:
def make_train(theta0):
    """Make an array of training data points over 10 years"""
    # Sample time at intervals of 365 per year (approximately daily)
    t = np.arange(3651, dtype=np.float32) / 365.0
    # Number of samples
    N = t.shape[0]
    # Set the angular frequency omega to that of earth
    omega = omega_earth
    # Theta is offset by theta0
    theta = omega * t - theta0

    # Compute x and y from theta; wrap into position q
    x = np.cos(theta, dtype=np.float32)
    y = np.sin(theta, dtype=np.float32)
    q = np.stack([x, y])
    
    # Compute vx and vy; wrap into velocity v
    vx = -omega * np.sin(theta, dtype=np.float32)
    vy = +omega * np.cos(theta, dtype=np.float32)
    v = np.stack([vx, vy])

    # Repeat vectors for the initial position and velocity
    x0 = np.ones(N) * x[0]
    y0 = np.ones(N) * y[0]
    vx0 = np.ones(N) * vx[0]
    vy0 = np.ones(N) * vy[0]
    
    # Assemble the input DataFrame
    df_input = pd.DataFrame()
    df_input['t'] = t
    df_input['x0'] = x0
    df_input['y0'] = y0
    df_input['vx0'] = vx0
    df_input['vy0'] = vy0

    # Assemble the output DataFrame
    df_output = pd.DataFrame()
    df_output['x'] = x
    df_output['y'] = y
    df_output['vx'] = vx
    df_output['vy'] = vy    
    
    # Return the DataFrames
    return (df_input, df_output)

In [129]:
df_input, df_output = make_train(0.0)

In [130]:
df_input[0:3]

,t,x0,y0,vx0,vy0
0,0.000000,1.0,0.0,-0.0,6.283185
1,0.002740,1.0,0.0,-0.0,6.283185
2,0.005479,1.0,0.0,-0.0,6.283185


In [131]:
df_output[0:365:30]

,x,y,vx,vy
0,1.000000,0.000000,-0.000000,6.283185
30,0.869589,0.493776,-3.102484,5.463791
60,0.512371,0.858764,-5.395773,3.219324
90,0.021516,0.999768,-6.281731,0.135189
120,-0.474951,0.880012,-5.529280,-2.984206
150,-0.847541,0.530730,-3.334674,-5.325257
180,-0.999074,0.043022,-0.270316,-6.277368
210,-0.890027,-0.455907,2.864547,-5.592207
240,-0.548843,-0.835926,5.252275,-3.448481
270,-0.064508,-0.997917,6.270099,-0.405318


In [165]:
batch_size = 32

# One input batch from DataFrame
t = tf.Variable(tf.reshape(df_input.t.iloc[0:batch_size], shape=(-1,1)), name='t')
x0 = tf.Variable(df_input.x0.iloc[0:batch_size], name='x0', dtype=tf.float32)
y0 = tf.Variable(df_input.y0.iloc[0:batch_size], name='y0', dtype=tf.float32)
vx0 = tf.Variable(df_input.vx0.iloc[0:batch_size], name='vx0', dtype=tf.float32)
vy0 = tf.Variable(df_input.vy0.iloc[0:batch_size], name='vy0', dtype=tf.float32)

# Stack position and velocity vectors
q0 = tf.Variable(tf.stack([x0, y0], axis=1))
v0 = tf.Variable(tf.stack([vx0, vy0], axis=1))

# Combine input values into one batch
inputs_batch = [t, q0, v0]

In [166]:
# Review time input
t

<tf.Variable 't:0' shape=(32, 1) dtype=float32, numpy=
array([[0.        ],
       [0.00273973],
       [0.00547945],
       [0.00821918],
       [0.0109589 ],
       [0.01369863],
       [0.01643836],
       [0.01917808],
       [0.02191781],
       [0.02465753],
       [0.02739726],
       [0.03013699],
       [0.03287671],
       [0.03561644],
       [0.03835616],
       [0.04109589],
       [0.04383562],
       [0.04657534],
       [0.04931507],
       [0.0520548 ],
       [0.05479452],
       [0.05753425],
       [0.06027397],
       [0.0630137 ],
       [0.06575342],
       [0.06849315],
       [0.07123288],
       [0.07397261],
       [0.07671233],
       [0.07945205],
       [0.08219178],
       [0.08493151]], dtype=float32)>

In [167]:
# Review initial position input
q0

<tf.Variable 'Variable:0' shape=(32, 2) dtype=float32, numpy=
array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)>

In [168]:
# Create a DataSet
ds = tf.data.Dataset.from_tensor_slices((dict(df_input), dict(df_output)))
ds

<TensorSliceDataset shapes: ({t: (), x0: (), y0: (), vx0: (), vy0: ()}, {x: (), y: (), vx: (), vy: ()}), types: ({t: tf.float32, x0: tf.float64, y0: tf.float64, vx0: tf.float64, vy0: tf.float64}, {x: tf.float32, y: tf.float32, vx: tf.float32, vy: tf.float32})>

In [169]:
# Example batch of inputs and outputs
for batch_in, batch_out in ds.take(1):
    pass

In [170]:
batch_in

{'t': <tf.Tensor: id=1243, shape=(), dtype=float32, numpy=0.0>,
 'x0': <tf.Tensor: id=1246, shape=(), dtype=float64, numpy=1.0>,
 'y0': <tf.Tensor: id=1247, shape=(), dtype=float64, numpy=0.0>,
 'vx0': <tf.Tensor: id=1244, shape=(), dtype=float64, numpy=-0.0>,
 'vy0': <tf.Tensor: id=1245, shape=(), dtype=float64, numpy=6.2831854820251465>}

In [171]:
batch_out

{'x': <tf.Tensor: id=1250, shape=(), dtype=float32, numpy=1.0>,
 'y': <tf.Tensor: id=1251, shape=(), dtype=float32, numpy=0.0>,
 'vx': <tf.Tensor: id=1248, shape=(), dtype=float32, numpy=-0.0>,
 'vy': <tf.Tensor: id=1249, shape=(), dtype=float32, numpy=6.2831855>}

In [172]:
batch_in['t']

<tf.Tensor: id=1243, shape=(), dtype=float32, numpy=0.0>

In [173]:
# Create input layers
input_t = keras.Input(shape=(1,), name='t')
input_q = keras.Input(shape=(2,), name='q')
input_v = keras.Input(shape=(2,), name='v')

In [174]:
# Review time input t
input_t

<tf.Tensor 't_2:0' shape=(None, 1) dtype=float32>

In [175]:
# Review position input q
input_q

<tf.Tensor 'q_2:0' shape=(None, 2) dtype=float32>

In [176]:
# Combine inputs
inputs = [input_t, input_q, input_v]
inputs

[<tf.Tensor 't_2:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'q_2:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'v_2:0' shape=(None, 2) dtype=float32>]

In [191]:
inputs_stack = tf.concat(inputs, axis=1)
inputs_stack

<tf.Tensor 'concat_15:0' shape=(None, 5) dtype=float32>

In [203]:
# The semi-major axis
orb_a = tf.keras.layers.Dense(1)(inputs_stack)

# The eccentricity
orb_e = tf.Variable(0.0, name='e')

# The angular frequency omega
omega = tf.Variable(omega_earth, name='omega')

# The angle theta (true anomaly)
theta = omega * input_t

# The semi-latus rectum
orb_p = orb_a * (1.0 - orb_e*orb_e)

# The radius
r = orb_p / (1 + orb_e * tf.cos(theta))

# The x and y position
x = r * tf.cos(theta)
y = r * tf.sin(theta)
q = tf.concat([x, y], axis=1)

# The x and y velocity
#TODO: update this later with automatic differentiation!
vx = omega * r * tf.cos(theta)
vy = omega * r * tf.sin(theta)
v = tf.concat([vx, vy], axis=1)

In [204]:
a

<tf.Tensor 'dense_6/Identity:0' shape=(None, 1) dtype=float32>

In [205]:
p

<tf.Tensor 'mul_61:0' shape=(None, 1) dtype=float32>

In [206]:
omega

<tf.Variable 'omega:0' shape=() dtype=float32, numpy=6.2831855>

In [207]:
theta

<tf.Tensor 'mul_69:0' shape=(None, 1) dtype=float32>

In [208]:
q

<tf.Tensor 'concat_18:0' shape=(None, 2) dtype=float32>

In [209]:
# Outputs
outputs = [q, v]
outputs

[<tf.Tensor 'concat_18:0' shape=(None, 2) dtype=float32>,
 <tf.Tensor 'concat_19:0' shape=(None, 2) dtype=float32>]

In [210]:
# Wrap model
# model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model = tf.keras.models.Model(inputs=inputs, outputs=a)

In [211]:
model(inputs)

<tf.Tensor 'model_6/Identity:0' shape=(None, 1) dtype=float32>

In [215]:
model.trainable_variables

[<tf.Variable 'dense_6/kernel:0' shape=(5, 1) dtype=float32, numpy=
 array([[ 0.04853988],
        [ 0.1174233 ],
        [-0.0816896 ],
        [ 0.5425074 ],
        [-0.4069805 ]], dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [212]:
model(inputs_batch)

AttributeError: in converted code:
    relative to C:\Anaconda3\lib\site-packages\tensorflow\python:

    keras\engine\base_layer.py:2446 _defun_call  *
        return self._make_op(inputs)
    keras\engine\base_layer.py:2413 _make_op
        graph = inputs[0].graph
    ops\resource_variable_ops.py:813 graph
        return self._handle.graph
    framework\ops.py:992 graph
        "Tensor.graph is meaningless when eager execution is enabled.")

    AttributeError: Tensor.graph is meaningless when eager execution is enabled.
